**K-Means : Algoritmo de Machine Learning para hacer Clustering en Sistemas de aprendizaje no supervizado**

**Utilizare el análisis de conglomerados para generar un modelo del clima  utilizando datos con definición de minuto. Este conjunto de datos tiene millones de registros. ¿Cómo creo 12 grupos?**

In [62]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from itertools import cycle, islice
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
import datetime

%matplotlib inline

In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
# Creo un Pandas DataFrame desde un archivo CSV
df = pd.read_csv('/content/drive/My Drive/minuto.csv')

 Estos datos provienen de una estación meteorológica. La estación meteorológica está equipada con sensores que capturan mediciones relacionadas con el clima, como la temperatura del aire, la presión del aire y la humedad relativa. Los datos se recopilaron durante un período de tres años, de septiembre de 2011 a septiembre de 2014, para garantizar que se capturan datos suficientes para diferentes estaciones y condiciones climáticas.

Cada fila  contiene datos meteorológicos capturados para un intervalo de un minuto. Cada fila, o muestra, consta de las siguientes variables:

* **rowID:** 	Llave única
* **hpwren_timestamp:**	timestamp de la medida	(*Unit: year-month-day hour:minute:second*)
* **air_pressure:** presión atmosférica	(*Hectopascales*)
* **air_temp:**	Temperatura del aire	(*Grados Fahrenheit*)
* **avg_wind_direction:**	direccion del viento promedio durante el minuto previo al timestamp
* **avg_wind_speed:**	velocidad del viento promedio durante el minuto previo al timestamp (metros por segundo)
* **max_wind_direction:**	Direccion máxima del viento
* **max_wind_speed:**	Velocidad máxima del viento
* **min_wind_direction:**	Dirección mínima del viento
* **min_wind_speed:**	Velocidad mínima del viento
* **rain_accumulation:**	Acumulación de lluvia al timestamp
* **rain_duration:**	duración de la lluvia
* **relative_humidity:**	humedad relativa medida en el timestamp **texto en negrita**

In [65]:
df.shape

(1587257, 13)

In [66]:
df.head()

,rowID,hpwren_timestamp,air_pressure,air_temp,avg_wind_direction,avg_wind_speed,max_wind_direction,max_wind_speed,min_wind_direction,min_wind_speed,rain_accumulation,rain_duration,relative_humidity
0,0,2011-09-10 00:00:49,912.3,64.76,97.0,1.2,106.0,1.6,85.0,1.0,NaN,NaN,60.5
1,1,2011-09-10 00:01:49,912.3,63.86,161.0,0.8,215.0,1.5,43.0,0.2,0.0,0.0,39.9
2,2,2011-09-10 00:02:49,912.3,64.22,77.0,0.7,143.0,1.2,324.0,0.3,0.0,0.0,43.0
3,3,2011-09-10 00:03:49,912.3,64.40,89.0,1.2,112.0,1.6,12.0,0.7,0.0,0.0,49.5
4,4,2011-09-10 00:04:49,912.3,64.40,185.0,0.4,260.0,1.0,100.0,0.1,0.0,0.0,58.8


In [67]:
# Tenemos demasiadas filas en el dataset, bajaremos la cantidad a un 10%
# Elijo una de cada 10 filas, para esto elijo cada fila que dividido 10 tiene resto 0
sampled_df = df[(df['rowID'] % 10) == 0]
sampled_df.shape

(158726, 13)

In [68]:
# Estadísticas
# Describo la muestra, paso columnas a filas y filas a columnas para que la descripción luzca mejor
sampled_df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
rowID,158726.0,793625.000000,458203.937509,0.00,396812.5,793625.00,1190437.50,1587250.00
air_pressure,158726.0,916.830161,3.051717,905.00,914.8,916.70,918.70,929.50
air_temp,158726.0,61.851589,11.833569,31.64,52.7,62.24,70.88,99.50
avg_wind_direction,158680.0,162.156100,95.278201,0.00,62.0,182.00,217.00,359.00
avg_wind_speed,158680.0,2.775215,2.057624,0.00,1.3,2.20,3.80,31.90
max_wind_direction,158680.0,163.462144,92.452139,0.00,68.0,187.00,223.00,359.00
max_wind_speed,158680.0,3.400558,2.418802,0.10,1.6,2.70,4.60,36.00
min_wind_direction,158680.0,166.774017,97.441109,0.00,76.0,180.00,212.00,359.00
min_wind_speed,158680.0,2.134664,1.742113,0.00,0.8,1.60,3.00,31.60
rain_accumulation,158725.0,0.000318,0.011236,0.00,0.0,0.00,0.00,3.12


In [69]:
# Veo  que las columnas rain accumulation y rain duration tienen valores en cero
sampled_df[sampled_df['rain_accumulation'] == 0].shape

(157812, 13)

In [70]:
sampled_df[sampled_df['rain_duration'] == 0].shape

(157237, 13)

**Limpio datos**

In [71]:
#como estas dos columnas tienen muchisimos ceros directamente las borro enteras
del sampled_df['rain_accumulation']
del sampled_df['rain_duration']

In [72]:
rows_before = sampled_df.shape[0]
#borro filas que contengan algun valor nulo
sampled_df = sampled_df.dropna()
rows_after = sampled_df.shape[0]

In [73]:
# Cuántas filas perdí?
rows_before - rows_after

46

In [74]:
sampled_df.columns #corroboro qué columnas quedaron

Index(['rowID', 'hpwren_timestamp', 'air_pressure', 'air_temp',
       'avg_wind_direction', 'avg_wind_speed', 'max_wind_direction',
       'max_wind_speed', 'min_wind_direction', 'min_wind_speed',
       'relative_humidity'],
      dtype='object')

**Selecciono las cualidades de interés para encarar el clustering**

In [75]:
# Estás son las columnas que seleccione como features
features = ['air_pressure', 'air_temp', 'avg_wind_direction', 'avg_wind_speed', 'max_wind_direction',
        'max_wind_speed','relative_humidity']

In [76]:
# filtro el contenido dejando solo las columnas seleccionadas
select_df = sampled_df[features]

In [77]:
select_df.columns

Index(['air_pressure', 'air_temp', 'avg_wind_direction', 'avg_wind_speed',
       'max_wind_direction', 'max_wind_speed', 'relative_humidity'],
      dtype='object')

**Scale the Features using StandardScaler**

In [78]:
X = StandardScaler().fit_transform(select_df)
X

array([[-1.48456281,  0.24544455, -0.68385323, ..., -0.62153592,
        -0.74440309,  0.49233835],
       [-1.48456281,  0.03247142, -0.19055941, ...,  0.03826701,
        -0.66171726, -0.34710804],
       [-1.51733167,  0.12374562, -0.65236639, ..., -0.44847286,
        -0.37231683,  0.40839371],
       ...,
       [-0.30488381,  1.15818654,  1.90856325, ...,  2.0393087 ,
        -0.70306017,  0.01538018],
       [-0.30488381,  1.12776181,  2.06599745, ..., -1.67073075,
        -0.74440309, -0.04948614],
       [-0.30488381,  1.09733708, -1.63895404, ..., -1.55174989,
        -0.62037434, -0.05711747]])

**Implementando k-Means Clustering**

In [79]:
# seteo la cantidad de clusters que quiero generar
kmeans = KMeans(n_clusters=12)
# Ajusto el modelo
model = kmeans.fit(X)
print("model\n", model)

model
 KMeans(n_clusters=12)


**El centro de los 12 grupos que forme**

In [80]:
centers = model.cluster_centers_
centers

array([[ 1.18604809, -0.2499142 , -1.15656178,  2.08215542, -1.05438855,
         2.19928092, -1.13211015],
       [-0.27034585,  0.56941502,  0.17429235, -0.68688155,  0.35578051,
        -0.6998217 , -0.39140786],
       [ 1.3924494 , -0.08837818, -1.125904  , -0.12434192, -0.98475888,
        -0.10601719, -0.97782534],
       [-1.46762349, -1.42855551,  0.38818168,  2.08967761,  0.49817913,
         2.05976066,  1.51924369],
       [-0.04719533,  0.72489093,  0.41718194,  0.46107871,  0.53242846,
         0.41862259, -0.25478512],
       [ 0.37864889, -0.96823856,  0.67264275, -0.53635673,  0.86704524,
        -0.51777677,  1.08170946],
       [-0.18185695,  0.86377575, -1.31236747, -0.57473764, -1.17088226,
        -0.59031663, -0.63206554],
       [-0.76784231, -0.18215053,  0.44364625,  1.56659189,  0.52810298,
         1.49724122,  0.26843102],
       [-0.86705585, -1.13525817,  0.36922532,  0.20386714,  0.47062284,
         0.19294122,  1.34540421],
       [ 0.27918343,  0.7755